In [1]:
import cv2
import darknet
import os
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt





In [2]:
cfg_file_path = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/chestxray.cfg"
data_file_path = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/chestxray.data"
weight_file_path = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/backup/chestxray_last.weights"
network,  class_names, class_colors = darknet.load_network(cfg_file_path, data_file_path, weight_file_path, batch_size=1)

darknet_width = darknet.network_width(network)
darknet_height = darknet.network_height(network)

print(darknet_width, darknet_height)

 Try to load cfg: /workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/chestxray.cfg, weights: /workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/backup/chestxray_last.weights, clear = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
Create CUDA-stream - 0 


 0 : compute_capability = 750, cudnn_half = 1, GPU: Quadro RTX 5000 
   layer   filters  size/strd(dil)      input                output
   0 

 Create cudnn-handle 0 


conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   9 route  8 2 	                           ->  304 x 304 x 128 
  10 conv     64       1 x 1/ 1    304 x 304 x 128 ->  304 x 304 x  64 1.514 BF
  11 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
  12 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
 

nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 


conv    512       3 x 3/ 1     38 x  38 x 256 ->   38 x  38 x 512 3.407 BF
 145 conv    256       1 x 1/ 1     38 x  38 x 512 ->   38 x  38 x 256 0.379 BF
 146 conv    512       3 x 3/ 1     38 x  38 x 256 ->   38 x  38 x 512 3.407 BF
 147 conv    256       1 x 1/ 1     38 x  38 x 512 ->   38 x  38 x 256 0.379 BF
 148 conv    512       3 x 3/ 1     38 x  38 x 256 ->   38 x  38 x 512 3.407 BF
 149 conv     57       1 x 1/ 1     38 x  38 x 512 ->   38 x  38 x  57 0.084 BF
 150 yolo
[yolo] params: iou loss: ciou (4), iou_norm: 0.07, obj_norm: 1.00, cls_norm: 1.00, delta_norm: 1.00, scale_x_y: 1.10
 151 route  147 		                           ->   38 x  38 x 256 
 152 conv    512       3 x 3/ 2     38 x  38 x 256 ->   19 x  19 x 512 0.852 BF
 153 route  152 116 	                           ->   19 x  19 x1024 
 154 conv    512       1 x 1/ 1     19 x  19 x1024 ->   19 x  19 x 512 0.379 BF
 155 conv   1024       3 x 3/ 1     19 x  19 x 512 ->   19 x  19 x1024 3.407 BF
 156 conv    512       

nms_kind: greedynms (1), beta = 0.600000 
 Try to load weights: /workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/backup/chestxray_last.weights 

 seen 64, trained: 649 K-images (10 Kilo-batches_64) 
608Loaded - names_list: /workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/chestxray.names, classes = 14 
 608


Done! Loaded 162 layers from weights-file 


In [3]:
DF_RAW = pd.read_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/EfficientNetB4/submission.csv")
DF_RAW.head()

,image_id,abnormal
0,26a5285339df294ce288663b092c23c3,1
1,31ccf1b8a9992b055d0d26043eea3c30,1
2,688ecdb1a4e994d42b5a50a8c4a9736f,1
3,73a47bff27e977dc25c0b2cd4a470eea,1
4,8bb8636f45e172949bb99b3d5dd715fd,1


In [4]:

def convert2relative(bbox):
    """
    YOLO format use relative coordinates for annotation
    """
    x, y, w, h  = bbox
    _height     = darknet_height
    _width      = darknet_width
    return x/_width, y/_height, w/_width, h/_height


def convert2original(image, bbox):
    x, y, w, h = convert2relative(bbox)

    image_h, image_w, __ = image.shape

    orig_x       = int(x * image_w)
    orig_y       = int(y * image_h)
    orig_width   = int(w * image_w)
    orig_height  = int(h * image_h)

    bbox_converted = (orig_x, orig_y, orig_width, orig_height)

    return bbox_converted


def convert4cropping(image, bbox):
    x, y, w, h = convert2relative(bbox)

    image_h, image_w, __ = image.shape

    orig_left    = int((x - w / 2.) * image_w)
    orig_right   = int((x + w / 2.) * image_w)
    orig_top     = int((y - h / 2.) * image_h)
    orig_bottom  = int((y + h / 2.) * image_h)

    if (orig_left < 0): orig_left = 0
    if (orig_right > image_w - 1): orig_right = image_w - 1
    if (orig_top < 0): orig_top = 0
    if (orig_bottom > image_h - 1): orig_bottom = image_h - 1

    bbox_cropping = (orig_left, orig_top, orig_right, orig_bottom)

    return bbox_cropping

In [5]:
# pbar = tqdm(range(20))
pbar = tqdm(range(len(DF_RAW)))
DF_SUBMISSION = pd.DataFrame(columns=["image_id", "PredictionString"])
for i in pbar:
    img_file = DF_RAW.iloc[i, 0] + ".png"
    img_path = os.path.join("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/DATA/dataset/test_png", img_file)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (darknet_width, darknet_height),
                                interpolation=cv2.INTER_LINEAR)
    
    img_for_detect = darknet.make_image(darknet_width, darknet_height, 3)
    darknet.copy_image_from_bytes(img_for_detect, img_resized.tobytes())
    detections = darknet.detect_image(network, class_names, img_for_detect, thresh=0.3)
    darknet.free_image(img_for_detect)
    new_string = ""
    if len(detections) != 0:
        for label, confidence, bbox in detections:
            # get coordinates of bounding box on raw image
            bbox = convert4cropping(img, bbox)
            x_min, y_min, x_max, y_max = bbox
            # get class id
            class_id = class_names.index(label)
            
            new_string += str(class_id) + " " + str(float(confidence)/100) + " " + str(x_min) + " " + str(y_min) + " " + str(x_max) + " " + str(y_max) + " "
    else:
        new_string = "14 1.0 0 0 1 1"   

    DF_SUBMISSION = DF_SUBMISSION.append({"image_id": DF_RAW.iloc[i, 0], "PredictionString": new_string}, ignore_index=True)

    


        
        
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    # plt.imshow(img)
    # plt.show()
    # break
    pbar.set_description("Processing {}".format(img_file))

Processing 547c3618a312f5c3cb3095d457483a57.png: 100%|██████████| 3000/3000 [05:39<00:00,  8.82it/s]


In [6]:
DF_SUBMISSION

,image_id,PredictionString
0,26a5285339df294ce288663b092c23c3,13 0.6231 2141 838 2237 893 13 0.8933 2092 618...
1,31ccf1b8a9992b055d0d26043eea3c30,14 1.0 0 0 1 1
2,688ecdb1a4e994d42b5a50a8c4a9736f,3 0.48460000000000003 679 1235 1656 1718
3,73a47bff27e977dc25c0b2cd4a470eea,4 0.33759999999999996 1320 2011 2128 2477 7 0....
4,8bb8636f45e172949bb99b3d5dd715fd,14 1.0 0 0 1 1
...,...,...
2995,f951f6a6d4fd2ac09f5e95a33c731a38,14 1.0 0 0 1 1
2996,3075e85dda12262bc9e34a5bd518d226,9 0.32130000000000003 706 2214 2674 2977
2997,40fb4c33b5cd086ae3eb5a38a6079fa2,8 0.5403 730 1330 1139 1861 4 0.5997 730 1330 ...
2998,9575166c507d7862f5924bc606eb6c1d,3 0.9786 741 1319 1597 1687 0 0.99400000000000...


In [7]:
# to csv
DF_SUBMISSION.to_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov4/darknet/submission_yolov4_14_cls_20000.csv", index=False)